In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DIRECTORY = "/content/drive/MyDrive/AIN413"

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

In [ ]:
!pip install -q -U datasets peft trl accelerate bitsandbytes transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.4/102.4 kB 15.1 MB/s eta 0:00:00


In [ ]:
from datasets import Dataset
from pathlib import Path
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from huggingface_hub import notebook_login
import torch

import warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [ ]:
#df = pd.read_parquet(Path(DIRECTORY) / "full_dataset.parquet")
prompts = pd.read_parquet(Path(DIRECTORY) / "prompts.parquet")

In [ ]:
#hf_df = Dataset.from_pandas(df, preserve_index=False)

hf_prompts = Dataset.from_pandas(prompts, preserve_index=False)
hf_prompts = hf_prompts.shuffle(seed=1)

In [ ]:
hf_prompts = hf_prompts.train_test_split(0.03)

In [ ]:
hf_prompts

DatasetDict({
    train: Dataset({
        features: ['prompt'],
        num_rows: 10825
    })
    test: Dataset({
        features: ['prompt'],
        num_rows: 335
    })
})

## Model Training

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)


model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto", low_cpu_mem_usage=True)
model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): Ll

In [ ]:
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj']
)

model = get_peft_model(model, peft_config)

In [ ]:
!pwd
!mkdir results

/content


In [ ]:
training_arguments = TrainingArguments(
        output_dir="./results", # repo name
        evaluation_strategy="steps",
        do_eval=True,
        optim="paged_adamw_32bit",
        learning_rate=2e-4,
        per_device_train_batch_size=8, # default
        gradient_accumulation_steps=4, # default
        per_device_eval_batch_size=8, # default
        eval_steps=20,
        save_steps=40,
        logging_steps=20,
        #max_steps=200,
        num_train_epochs=1,
        warmup_ratio=0.1,
        lr_scheduler_type="linear",
)

In [ ]:
trainer =  SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=hf_prompts["train"],
    dataset_text_field="prompt",
    eval_dataset=hf_prompts["test"],
    tokenizer=tokenizer,
    max_seq_length=1024
)

trainer.train()

Map:   0%|          | 0/10825 [00:00<?, ? examples/s]

Map:   0%|          | 0/335 [00:00<?, ? examples/s]

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
20,2.217000,1.587596
40,1.482100,1.364877
60,1.321700,1.250070
80,1.239200,1.220144
100,1.196300,1.207533
120,1.182900,1.199718
140,1.222900,1.191693
160,1.201600,1.186755
180,1.175300,1.183065
200,1.216000,1.178972


TrainOutput(global_step=338, training_loss=1.2843232747365738, metrics={'train_runtime': 15621.3674, 'train_samples_per_second': 0.693, 'train_steps_per_second': 0.022, 'total_flos': 2.1992763882523853e+17, 'train_loss': 1.2843232747365738, 'epoch': 0.9985228951255539})

### Saving Model

In [ ]:
!mkdir model

In [ ]:
trainer.save_model("./model")

In [ ]:
from google.colab import userdata

trainer.push_to_hub("ae-aydin/Llama-3-8B-Instruct-Medical-Finetune", token=userdata.get('write'))

events.out.tfevents.1715888893.28d91cbc4c1e.1997.0:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/83.9M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.98k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ae-aydin/results/commit/6a7a8d9b32845c8af586e205e64b224a4a807eeb', commit_message='ae-aydin/Llama-3-8B-Instruct-Medical-Finetune', commit_description='', oid='6a7a8d9b32845c8af586e205e64b224a4a807eeb', pr_url=None, pr_revision=None, pr_num=None)

***

In [ ]:
# plug adapter
#base_model = AutoModelForCausalLM.from_pretrained(default_model_id, quantization_config=bnb_config, low_cpu_mem_usage=True)
#peft_config = PeftConfig.from_pretrained(finetuned_adapter_id)
#model = PeftModel.from_pretrained(base_model, finetuned_adapter_id)

# Merge adapter and model
#model = model.merge_and_unload()
#model.save_pretrained(save_directory=finetuned_model_id)

# Load adapter
#model.load_adapter(finetuned_adapter_id, adapter_name="medical")
#model.set_adapter("medical")